In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from datetime import datetime as dts
from tqdm import tqdm
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
tqdm.pandas()
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/child-mind-institute-detect-sleep-states/train_series.parquet
/kaggle/input/child-mind-institute-detect-sleep-states/sample_submission.csv
/kaggle/input/child-mind-institute-detect-sleep-states/train_events.csv
/kaggle/input/child-mind-institute-detect-sleep-states/test_series.parquet


In [2]:
df_series=pd.read_parquet(path="/kaggle/input/child-mind-institute-detect-sleep-states/train_series.parquet", engine='auto')

In [3]:
df_events=pd.read_csv("/kaggle/input/child-mind-institute-detect-sleep-states/train_events.csv")

In [4]:
def data_preparation(df):
    df['timestamp'] = pd.to_datetime(df['timestamp'], utc=True,)
    df['step']=pd.to_numeric(df['step'])
    print('.',end="")
    return(df)


In [5]:
df_events.head()

,series_id,night,event,step,timestamp
0,038441c925bb,1,onset,4992.0,2018-08-14T22:26:00-0400
1,038441c925bb,1,wakeup,10932.0,2018-08-15T06:41:00-0400
2,038441c925bb,2,onset,20244.0,2018-08-15T19:37:00-0400
3,038441c925bb,2,wakeup,27492.0,2018-08-16T05:41:00-0400
4,038441c925bb,3,onset,39996.0,2018-08-16T23:03:00-0400


In [6]:
df_events=df_events.groupby('series_id').progress_apply(lambda x: data_preparation(x))
df_events.info()

 34%|███▍      | 95/277 [00:00<00:00, 481.31it/s]

...................................................................................................................

 74%|███████▎  | 204/277 [00:00<00:00, 523.38it/s]

................................................................................................................

100%|██████████| 277/277 [00:00<00:00, 442.25it/s]

..................................................<class 'pandas.core.frame.DataFrame'>
MultiIndex: 14508 entries, ('038441c925bb', 0) to ('fe90110788d2', 14507)
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype              
---  ------     --------------  -----              
 0   series_id  14508 non-null  object             
 1   night      14508 non-null  int64              
 2   event      14508 non-null  object             
 3   step       9585 non-null   float64            
 4   timestamp  9585 non-null   datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](1), float64(1), int64(1), object(2)
memory usage: 1.2+ MB


In [7]:
df_series.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127946340 entries, 0 to 127946339
Data columns (total 5 columns):
 #   Column     Dtype  
---  ------     -----  
 0   series_id  object 
 1   step       uint32 
 2   timestamp  object 
 3   anglez     float32
 4   enmo       float32
dtypes: float32(2), object(2), uint32(1)
memory usage: 3.3+ GB


In [8]:
df_series=df_series.groupby('series_id').count()

In [9]:
df_series=df_series.groupby('series_id').progress_apply(lambda x: data_preparation(x))
df_series.info()

100%|██████████| 277/277 [00:00<00:00, 1511.38it/s]

.....................................................................................................................................................................................................................................................................................<class 'pandas.core.frame.DataFrame'>
MultiIndex: 277 entries, ('038441c925bb', '038441c925bb') to ('fe90110788d2', 'fe90110788d2')
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype              
---  ------     --------------  -----              
 0   step       277 non-null    int64              
 1   timestamp  277 non-null    datetime64[ns, UTC]
 2   anglez     277 non-null    int64              
 3   enmo       277 non-null    int64              
dtypes: datetime64[ns, UTC](1), int64(3)
memory usage: 30.4+ KB


In [10]:
df_series.head()

,,step,timestamp,anglez,enmo
series_id,series_id,,,,
038441c925bb,038441c925bb,389880,1970-01-01 00:00:00.000389880+00:00,389880,389880
03d92c9f6f8a,03d92c9f6f8a,724140,1970-01-01 00:00:00.000724140+00:00,724140,724140
0402a003dae9,0402a003dae9,397260,1970-01-01 00:00:00.000397260+00:00,397260,397260
04f547b8017d,04f547b8017d,637560,1970-01-01 00:00:00.000637560+00:00,637560,637560
05e1944c3818,05e1944c3818,400860,1970-01-01 00:00:00.000400860+00:00,400860,400860
